<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/pooling_lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [2]:
import numpy as np
import pandas as pd

from ddop.newsvendor import LinearRegressionNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from ddop.metrics import average_costs
from ddop.metrics import prescriptiveness_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## loading data-set


---



*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**

In [4]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## split in feature and target matrix


---



In [5]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split


---



In [6]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## new order after the split


---



*   for the product pooling the whole data must be be sorted by date independent from the 'item_id'
*   therefore sort values by 'date' in the first instance and by 'item_id' in the second 
*   these two instances create the mutli-index of the target matrix (Y)
*   as all items are in one set now the 'item_id' becomes a feature in the feature matrix (X). The only index is the date here.







In [7]:
X_train = X_train.reset_index().sort_values(by=['date', 'item_id']).set_index(['date'])
X_test = X_test.reset_index().sort_values(by=['date', 'item_id']).set_index(['date'])

Y_train = Y_train.reset_index().sort_values(by=['date', 'item_id']).set_index(['date', 'item_id'])
Y_test = Y_test.reset_index().sort_values(by=['date', 'item_id']).set_index(['date', 'item_id'])

## defining the different cost paramerters for special service level and product range



---



*   cu is the mean of the cumulated 'sell_price' column of the training data
*   appending to the corresponding service level co is calculated with (cu - α * cu) / α
*   all the costs are rounded to 2 digits







In [8]:
prods = list(range(0,25))

cu = round(X_train['sell_price'].mean(),2)

co_90 = round(((cu-(0.90*cu))/0.90),2)

co_75 = round(((cu-(0.75*cu))/0.75),2)

co_50 = round(((cu-(0.50*cu))/0.50),2)

# newsvendor-model 90%



---



*   the model is trained and tested once for entire data-set
*   the prediction is the output



In [9]:
lr = LinearRegressionNewsvendor(cu = cu, co = co_90)

lr.fit(X_train, Y_train)

preds_90 = lr.predict(X_test)

In [21]:
preds_90

array([[22.99999998],
       [ 4.        ],
       [18.99999998],
       ...,
       [31.99999997],
       [34.99999997],
       [22.99999998]])

## reorder the data




---



*   create a pandas frame 'comp_90' where the real values, the predictions and the predictions from the saa model are in columns next to each other






In [10]:
pred_90 = pd.DataFrame(preds_90, columns=['pred_lr_90'])
comp_90 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_90 = comp_90.reset_index()
comp_90['pred_lr_90'] = pred_90[['pred_lr_90']]
comp_90 = comp_90.sort_values(by=['item_id', 'date']).set_index(['item_id', 'date'])

saa_90 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_90.csv').set_index(['item_id', 'date'])

comp_90 = pd.concat([comp_90, saa_90], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame






In [12]:
result_lr_90_avc = []

for prod in prods:
  avc = round(average_costs(comp_90['demand'].loc[prod], comp_90['pred_lr_90'].loc[prod], cu = cu, co = co_90),2)
  result_lr_90_avc.append(avc)

In [13]:
LR_pooled_avc_90 = pd.DataFrame(result_lr_90_avc, columns=['LR_avc_90'])
LR_pooled_avc_90.index.name = 'item_id'

In [15]:
result_lr_90_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_90['demand'].loc[prod], comp_90['pred_lr_90'].loc[prod],
                                      comp_90['SAA_pred_90'].loc[prod], cu = cu, co = co_90),2)
  result_lr_90_pscr.append(pscr)

In [16]:
LR_pooled_pscr_90 = pd.DataFrame(result_lr_90_pscr, columns=['LR_pscr_90'])
LR_pooled_pscr_90.index.name = 'item_id'

# newsvendor-model 75%



---



*   the model is trained and tested once for entire data-set
*   the prediction is the output

In [18]:
lr = LinearRegressionNewsvendor(cu = cu, co = co_75)

lr.fit(X_train, Y_train)

preds_75 = lr.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_75' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [19]:
pred_75 = pd.DataFrame(preds_75, columns=['pred_lr_75'])
comp_75 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_75 = comp_75.reset_index()
comp_75['pred_lr_75'] = pred_75[['pred_lr_75']]
comp_75 = comp_75.set_index(['item_id', 'date'])

saa_75 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_75.csv').set_index(['item_id', 'date'])

comp_75 = pd.concat([comp_75, saa_75], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [22]:
result_lr_75_avc = []

for prod in prods:
  avc = round(average_costs(comp_75['demand'].loc[prod], comp_75['pred_lr_75'].loc[prod], cu = cu, co = co_75),2)
  result_lr_75_avc.append(avc)

In [23]:
LR_pooled_avc_75 = pd.DataFrame(result_lr_75_avc, columns=['LR_avc_75'])
LR_pooled_avc_75.index.name = 'item_id'

In [25]:
result_lr_75_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_75['demand'].loc[prod], comp_75['pred_lr_75'].loc[prod],
                                      comp_75['SAA_pred_75'].loc[prod], cu = cu, co = co_75),2)
  result_lr_75_pscr.append(pscr)

In [26]:
LR_pooled_pscr_75 = pd.DataFrame(result_lr_75_pscr, columns=['LR_pscr_75'])
LR_pooled_pscr_75.index.name = 'item_id'

# newsvendor-model 50%



---



*   the model is trained and tested once for entire data-set
*   the prediction is the output

In [27]:
lr = LinearRegressionNewsvendor(cu = cu, co = co_50)

lr.fit(X_train, Y_train)

preds_50 = lr.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_50' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [28]:
pred_50 = pd.DataFrame(preds_50, columns=['pred_lr_50'])
comp_50 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_50 = comp_50.reset_index()
comp_50['pred_lr_50'] = pred_50[['pred_lr_50']]
comp_50 = comp_50.set_index(['item_id', 'date'])

saa_50 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_50.csv').set_index(['item_id', 'date'])

comp_50 = pd.concat([comp_50, saa_50], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [29]:
result_lr_50_avc = []

for prod in prods:
  avc = round(average_costs(comp_50['demand'].loc[prod], comp_50['pred_lr_50'].loc[prod], cu = cu, co = co_50),2)
  result_lr_50_avc.append(avc)

In [30]:
LR_pooled_avc_50 = pd.DataFrame(result_lr_50_avc, columns=['LR_avc_50'])
LR_pooled_avc_50.index.name = 'item_id'

In [31]:
result_lr_50_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_50['demand'].loc[prod], comp_50['pred_lr_50'].loc[prod],
                                      comp_50['SAA_pred_50'].loc[prod], cu = cu, co = co_50),2)
  result_lr_50_pscr.append(pscr)

In [32]:
LR_pooled_pscr_50 = pd.DataFrame(result_lr_50_pscr, columns=['LR_pscr_50'])
LR_pooled_pscr_50.index.name = 'item_id'

# merge the pandas frames


---



In [33]:
LR_pooled_avc = pd.concat([LR_pooled_avc_90, LR_pooled_avc_75, LR_pooled_avc_50], axis=1)
LR_pooled_avc

,LR_avc_90,LR_avc_75,LR_avc_50
item_id,,,
0,2.11,4.96,8.45
1,0.88,2.06,3.60
2,2.01,4.57,7.54
3,1.57,3.81,7.06
4,2.00,4.39,8.53
5,1.15,2.52,4.74
6,1.48,3.61,6.70
7,1.37,3.47,6.40
8,1.08,2.63,4.58


In [34]:
LR_pooled_pscr = pd.concat([LR_pooled_pscr_90, LR_pooled_pscr_75, LR_pooled_pscr_50], axis=1)
LR_pooled_pscr

,LR_pscr_90,LR_pscr_75,LR_pscr_50
item_id,,,
0,0.43,0.34,0.33
1,0.51,0.48,0.43
2,0.31,0.19,0.19
3,0.70,0.63,0.54
4,0.38,0.29,0.23
5,0.45,0.40,0.26
6,0.56,0.49,0.44
7,0.50,0.28,0.20
8,0.44,0.29,0.29


# save the results


---



In [35]:
LR_pooled_avc.to_csv('/content/drive/MyDrive/M5/LR_pooled_avc.csv')

LR_pooled_pscr.to_csv('/content/drive/MyDrive/M5/LR_pooled_pscr.csv')